In [2]:
import pandas as pd
import praw
from datetime import datetime
from time import sleep
from dotenv import load_dotenv
import os

load_dotenv()

# Authentication details from Reddit Developer Console
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = os.getenv("REDDIT_USER_AGENT")

# Initialize PRAW Reddit instance
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

print(f"Logged in as: {reddit.user.me()}")  # Verify connection: output should be "None" for read-only access

Logged in as: None


In [3]:
# Testing Purpose
for post in reddit.subreddit("UIUC").new(limit=1):
    # post.comments.replace_more(limit=0)
    # [print(comment.body) for comment in post.comments.list()[:10]]
    # print(post.comments.list().body)
    print(datetime.fromtimestamp(post.created_utc).isoformat())

2024-12-04T09:40:07


In [4]:
subreddit_name = "UIUC"
subreddit = reddit.subreddit(subreddit_name)

limit = 100  # Number of posts to scrape
posts = []

for post in subreddit.new(limit=limit):  # Change 'hot' to 'new', 'top', etc., as needed
    post.comments.replace_more(limit=0)

    posts.append({
        "title": post.title,
        "author": post.author.name if post.author else "deleted",
        "upvote": post.score,
        "body": post.selftext if post.selftext else "",
        "comments": [
            {
                "author": comment.author.name if comment.author else "deleted",
                "body": comment.body if comment.body else "",
                "created_at": datetime.fromtimestamp(comment.created_utc).isoformat()
            } for comment in post.comments.list()],
        "id": post.name,
        "url": post.url,
        "permalink": post.permalink,
        "created_at": datetime.fromtimestamp(post.created_utc).isoformat(),
    })
    sleep(1)

df = pd.DataFrame(posts)
df.to_csv("uiuc_subreddit_posts_2.csv", index=False)
print("posts saved to uiuc_subreddit_posts.csv")

posts saved to uiuc_subreddit_posts.csv


In [10]:
df = pd.read_csv("uiuc_subreddit_posts_2.csv")
pd.set_option('display.max_colwidth', None)
df.shape

(100, 9)

In [11]:
df.head()

,title,author,upvote,body,comments,id,url,permalink,created_at
0,Looking for roommates for next year,Ok_Board5095,1,"Hello! I'm a freshman looking to get an apartment next year. I'm looking at a few places, but they all have 2 or more bedrooms.\n\nPlease dm me if you are also looking for a roomate for next year!\n\nI'm mainly looking for women, as a women myself, but I'm open to men too.",[],t3_1h6j0d1,https://www.reddit.com/r/UIUC/comments/1h6j0d1/looking_for_roommates_for_next_year/,/r/UIUC/comments/1h6j0d1/looking_for_roommates_for_next_year/,2024-12-04T09:40:07
1,"Frosted Flakes w/ hot dog for breakfast. Remember, having a good breakfast helps performance in school.",AngusHornfeck,8,NaN,"[{'author': 'sjk8990', 'body': 'I think I just witnessed a war crime.', 'created_at': '2024-12-04T09:21:54'}, {'author': 'no-temperature-5932', 'body': ""Tough times won't last forever. Good luck!"", 'created_at': '2024-12-04T09:21:38'}, {'author': 'ItsTheOrangShep', 'body': ""It's weird. And probably disgusting.\n\nBut I must respect it.\n\nYou are a madlad.\n\nTake an upvote."", 'created_at': '2024-12-04T09:26:29'}, {'author': 'AngusHornfeck', 'body': ""For those of you eating your breakfast now. Here's an article to read while you eat: C:\\Users\\Angus\\Desktop\\RESEARCH\\2024\\November\\New folder\\UIUCFRAUD.pdf"", 'created_at': '2024-12-04T09:29:54'}, {'author': 'AHMS_17', 'body': 'Type shit', 'created_at': '2024-12-04T09:50:42'}]",t3_1h6ifly,https://i.redd.it/26xg0fj6mu4e1.png,/r/UIUC/comments/1h6ifly/frosted_flakes_w_hot_dog_for_breakfast_remember/,2024-12-04T09:16:32
2,FREEZINGGGGG 🥶,Initial_Hunt984,1,Is any one else freezing in their dorm??? Like the heating isn’t high enough to warm up the room. Like I have to sleep with double blankets last night. Also housing said they would turn it up when it drops below a certain degree and I’m pretty sure the heat should be blasting by now.(I’m in Taft btw),[],t3_1h6hz1q,https://www.reddit.com/r/UIUC/comments/1h6hz1q/freezinggggg/,/r/UIUC/comments/1h6hz1q/freezinggggg/,2024-12-04T08:57:43
3,Selling Meal Tickets,Financial-Size9431,1,Meal tickets $6 or could arrange a deal if buying a lot. Expiration: end of spring semester. Union pickup,[],t3_1h6hhgt,https://www.reddit.com/r/UIUC/comments/1h6hhgt/selling_meal_tickets/,/r/UIUC/comments/1h6hhgt/selling_meal_tickets/,2024-12-04T08:36:28
4,"TODAY! Get ready to DANCE! Krannert Center is having their annual Dance Break, and YOU are invited! Join us Wed. 4 Dec. from 12:05 - 12:20pm! Take a break and let some of that end-of-semester stress go! We'll be doin' some movin' in the Krannert Center lobby.",CMI_notes,1,NaN,[],t3_1h6fvml,https://www.instagram.com/reel/DDH-RcqhU4U/,/r/UIUC/comments/1h6fvml/today_get_ready_to_dance_krannert_center_is/,2024-12-04T07:23:32
